In [1]:
### Import packages ###
import time
import numpy as np
import math as math
import pandas as pd
import random as random

### Import functions ###
from utils.Main import *
from utils.Selector import *
from utils.Auxiliary import *
from utils.Prediction import *

In [2]:
### Get Directory ###
cwd = os.getcwd()
ParentDirectory = os.path.abspath(os.path.join(cwd, ".."))

### DataType ###
DataFileInput = "BostonHousingBinned"

# One Iteration

## Inputs

In [3]:
SimulationConfigInput = {"DataFileInput": 'BostonHousingBinned' ,
                         "Seed": 0,
                         "TestProportion" : 0.2,
                         "CandidateProportion" : 0.8,
                         "SelectorType" : "TreeEnsembleQBCFunction", 
                         "ModelType" : "TreeFarmsFunction", 
                         "TopCModels": 5, 
                         "UniqueErrorsInput": 1,
                         "n_estimators": 100, 
                         "regularization": 0.01,
                         "rashomon_bound_adder": 0.02,
                         "Type": "Classification"
                         }

In [4]:
# TopCModels = 50
# Seed = 1
# TestProportion = 0.2
# CandidateProportion = 0.8
# SelectorType = TreeEnsembleQBCFunction
# # ModelType = RandomForestClassificationFunction
# ModelType = TreeFarmsFunction
# DataArgs = {}
# SelectorArgs = {"TopCModels": 10}
# ModelArgs = {"TopCModels": TopCModels, 
#              "config" : {"regularization": 0.01, 
#              "rashomon_bound_adder": 0.05}, 
#              "Seed": Seed,
#              "n_estimators": TopCModels,
#              "Type":"Classification"}


In [5]:
# ### Parameter Vector ###
# k=3
# ParameterVector = pd.read_csv(os.path.join(ParentDirectory, "Data", "ParameterVectors", "ParameterVector" + DataFileInput + ".csv"))
# Seed = int(ParameterVector.iloc[k]["Seed"])
# TestProportion = float(ParameterVector.iloc[k]["TestProportion"])
# CandidateProportion = float(ParameterVector.iloc[k]["CandidateProportion"])
# SelectorType = globals().get(ParameterVector.iloc[k]["SelectorType"], None)
# # ModelType = globals().get(ParameterVector.iloc[k]["ModelType"], None)
# ModelType = RandomForestClassificationFunction
# DataArgs = ast.literal_eval(ast.literal_eval(ParameterVector.iloc[k]["DataArgs"]))
# SelectorArgs = ast.literal_eval(ast.literal_eval(ParameterVector.iloc[k]["SelectorArgs"].replace("[","").replace("]","")))
# ModelArgs = ast.literal_eval(ast.literal_eval(ParameterVector.iloc[k]["ModelArgs"].replace("[","").replace("]","")))


# Set Up

In [6]:
### Run Time ###
StartTime = time.time()

### Set Up ###
random.seed(SimulationConfigInput["Seed"])
np.random.seed(SimulationConfigInput["Seed"])
ErrorVec = []
SelectedObservationHistory = []

In [7]:
### Generate Data ###
# if(DataFileInput == "Simulate"):
#     from utils.Main import DataGeneratingProcess                             ### NOTE: Why is this not imported from utils.Main import *
#     df = DataGeneratingProcess(**DataArgs)
# else:
df = LoadData(SimulationConfigInput["DataFileInput"])


In [8]:
### Train Test Candidate Split ###
from utils.Main import TrainTestCandidateSplit                           ### NOTE: Why is this not imported from utils.Main import *
df_Train, df_Test, df_Candidate = TrainTestCandidateSplit(df, SimulationConfigInput["TestProportion"], SimulationConfigInput["CandidateProportion"])


In [9]:
### Update SimulationConfig Arguments ###
SimulationConfigInput['df_Train'] = df_Train
SimulationConfigInput["df_Test"] = df_Test                                     # NOTE: Change to df_Test if there is a test set
SimulationConfigInput["df_Candidate"] = df_Candidate

# Learning Procedure

In [10]:
### Input ###
SimulationConfigInputUpdated = SimulationConfigInput
 
### Set Up ###
ErrorVec = []
SelectedObservationHistory = []

In [11]:
i=0

In [12]:
### Prediction Model ###
print("Iteration: " + str(i))
ModelType = globals().get(SimulationConfigInputUpdated["ModelType"], None)
ModelArgsFiltered = FilterArguments(ModelType, SimulationConfigInputUpdated)
Model = ModelType(**ModelArgsFiltered)
SimulationConfigInputUpdated['Model'] = Model

Iteration: 0
null
Finding Optimal Objective...
treefarms reported successful execution
training completed. Number of trees in the Rashomon set: 2497
{
  "false": {
    "false": {
      "complexity": 0.009999999776482582,
      "loss": 0.012500000186264515,
      "name": "Y",
      "prediction": "False"
    },
    "feature": 8,
    "name": "RM_3",
    "reference": 1.0,
    "relation": "==",
    "true": {
      "false": {
        "false": {
          "complexity": 0.009999999776482582,
          "loss": 0.0,
          "name": "Y",
          "prediction": "False"
        },
        "feature": 5,
        "name": "NOX_3",
        "reference": 1.0,
        "relation": "==",
        "true": {
          "complexity": 0.009999999776482582,
          "loss": 0.0,
          "name": "Y",
          "prediction": "True"
        },
        "type": "rational"
      },
      "feature": 12,
      "name": "LSTAT_1",
      "reference": 1.0,
      "relation": "==",
      "true": {
        "complexity": 0.0

In [13]:
### Current Error ###
TestErrorVal = TestErrorFunction(Model, SimulationConfigInputUpdated["df_Test"], SimulationConfigInputUpdated["Type"])
if(len(TestErrorVal) > 1):
    AllErrors = TestErrorVal                                                # Rashomon gives all errors of Rashomon
    CurrentError = float(np.min(AllErrors))                                 # Extract the best one
else: 
    CurrentError = TestErrorVal                                             # One output for non-Rashomon
    AllErrors = [None]
SimulationConfigInputUpdated["AllErrors"] = AllErrors                              # Use AllErrors in RashomonQBC
ErrorVec.append(CurrentError)


In [14]:
### Sampling Procedure ###
SelectorType = globals().get(SimulationConfigInputUpdated["SelectorType"], None)
SelectorArgsFiltered = FilterArguments(SelectorType, SimulationConfigInputUpdated)
QueryObservationIndex = SelectorType(**SelectorArgsFiltered)
QueryObservation = df_Candidate.loc[[QueryObservationIndex]] # or should this be iloc

In [15]:
df_Candidate

,Y,CRIM_1,CRIM_2,CRIM_3,NOX_1,NOX_2,NOX_3,RM_1,RM_2,RM_3,PTRATIO_1,PTRATIO_2,PTRATIO_3,LSTAT_1,LSTAT_2,LSTAT_3
264,True,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
43,False,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
51,False,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
401,False,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
256,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,False,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
214,False,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
44,False,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
335,False,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [16]:
SimulationConfigInputUpdated["df_Candidate"]

,Y,CRIM_1,CRIM_2,CRIM_3,NOX_1,NOX_2,NOX_3,RM_1,RM_2,RM_3,PTRATIO_1,PTRATIO_2,PTRATIO_3,LSTAT_1,LSTAT_2,LSTAT_3
264,True,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
43,False,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
51,False,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
401,False,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
256,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,False,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
214,False,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
44,False,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
335,False,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [17]:
### Update Train and Candidate Sets ###
df_Train = pd.concat([df_Train, QueryObservation])
df_Candidate = SimulationConfigInputUpdated["df_Candidate"].drop(QueryObservationIndex)

### Update SimulationConfigInputUpdated ###
SimulationConfigInputUpdated['df_Train'] = df_Train
SimulationConfigInputUpdated['df_Candidate'] = df_Candidate  

In [24]:
SimulationParameters = {"DataFileInput" : str(SimulationConfigInput["DataFileInput"]),
                            "Seed" : str(SimulationConfigInput["Seed"]),
                            "TestProportion" : str(SimulationConfigInput["TestProportion"]),
                            "CandidateProportion" : str(SimulationConfigInput["CandidateProportion"]),
                            "SelectorType" :  str(SimulationConfigInput["SelectorType"]),
                            "ModelType" :  str(SimulationConfigInput["ModelType"]),
                            'UniqueErrorsInput': str(SimulationConfigInput["UniqueErrorsInput"]),
                            'n_estimators': str(SimulationConfigInput["n_estimators"]),
                            'regularization': str(SimulationConfigInput["regularization"]),
                            'rashomon_bound_adder': str(SimulationConfigInput["rashomon_bound_adder"]),
                            'Type': 'Classification',
                            }

# Save

In [ ]:
# ### Return Simulation Parameters ###
# SimulationParameters = {"DataFileInput" : str(DataFileInput),
#                         "Seed" : str(Seed),
#                         "TestProportion" : str(TestProportion),
#                         "CandidateProportion" : str(CandidateProportion),
#                         "SelectorType" : str(SelectorType),
#                         "ModelType" : str(ModelType),
#                         "DataArgs" : str(DataArgs),
#                         # "SelectorArgs" : str(SelectorArgs),
#                         "ModelArgs" : str(FilterArguments(ModelType, ModelArgs).pop('df_Train', None))
#                         }

In [ ]:
# ### Return Time ###
# ElapsedTime = time.time() - StartTime

# ### Return Dictionary ###
# SimulationResults = {"ErrorVec" : pd.DataFrame(ErrorVec, columns =["Error"]),
#                             "SelectionHistory" : pd.DataFrame(SelectedObservationHistory, columns = ["ObservationID"]),
#                             "SimulationParameters" : SimulationParameters,
#                             "ElapsedTime" : ElapsedTime}